# 2.5 KPIs Avanzados\n\n- Objetivo: derivar KPIs (ticket, margen, crecimiento).\n- Prerrequisitos: ventas y productos con precios/costos.\n- Ejercicios: ticket promedio, margen por segmento, crecimiento diario.\n- Reto: retención aproximada por clientes.\n- Errores comunes: COUNT vs COUNT(DISTINCT), divisiones por cero, agrupar tras calcular.

In [ ]:
-- Margen promedio por producto
SELECT p.producto_id, AVG(p.precio_unitario - p.costo_unitario) AS margen_promedio
FROM dbo.dim_productos p
GROUP BY p.producto_id;

In [ ]:
-- Crecimiento de unidades día a día (window + expresión)
WITH diario AS (
  SELECT fecha, SUM(cantidad) AS unidades
  FROM dbo.fact_ventas
  GROUP BY fecha
)
SELECT fecha, unidades,
       unidades - LAG(unidades) OVER (ORDER BY fecha) AS delta_unidades,
       CASE WHEN LAG(unidades) OVER (ORDER BY fecha) = 0 THEN NULL
            ELSE CAST((unidades - LAG(unidades) OVER (ORDER BY fecha)) AS DECIMAL(10,2)) / NULLIF(LAG(unidades) OVER (ORDER BY fecha),0) END AS pct_crecimiento
FROM diario;

🟢 Ejercicio: Ticket promedio (ingresos / transacciones).

In [ ]:
SELECT CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_promedio
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id;

🟠 Ejercicio: Margen total y participación porcentual por segmento.

In [ ]:
WITH margen_segmento AS (
  SELECT c.segmento, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY c.segmento
), total AS (SELECT SUM(margen) AS mt FROM margen_segmento)
SELECT segmento, margen, CAST(margen/mt*100 AS DECIMAL(5,2)) AS pct
FROM margen_segmento CROSS JOIN total;

🔴 Reto: Calcular retención aproximada: clientes con ventas en al menos 2 días distintos / total clientes activos.

Errores comunes: dividir antes de agrupar; usar COUNT(*) cuando se requiere COUNT(DISTINCT); no manejar divisiones por cero con NULLIF.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.